# The Evolution of LangChain Agents: From Classic ReAct to Modern Tool Calling

This notebook is a deep dive into the practical realities of building AI Agents with LangChain. It documents the evolution of agent creation, starting from the foundational, text-based ReAct framework and culminating in the modern, robust, and reliable Tool Calling architecture.

</br>
The central challenge we explore is a common one for AI engineers: how do we get reliable, well-structured, and debuggable output from an LLM that is fundamentally probabilistic? This notebook is a hands-on journey through that very problem.

</br>
We will build the same simple "time-telling" agent in three distinct stages, each in its own cell:

</br>

1. **The Classic create_react_agent**: We begin with the original, foundational method for building agents in LangChain. By setting verbose=True, we will immediately see the power of the ReAct reasoning loop, but also witness its primary weakness: the messy, sometimes inconsistent, text-based output that can be difficult to parse and debug.
2. **The Engineering Fix (Callbacks & Custom Runners)**: Faced with the messy output from the classic agent, our next step is a common engineering impulse: to try and fix the problem with more code. This cell demonstrates an attempt to wrestle control over the output formatting by using custom LangChain callbacks. This is a valuable exercise in understanding the framework's internals, but it ultimately shows the difficulty of patching a system that is inherently text-based.
3. **The Modern Solution (create_openai_tools_agent)**: Finally, we arrive at the absolute fix. This cell demonstrates the modern, industry-standard approach using native Tool Calling. Instead of parsing text, this method relies on the LLM generating a structured JSON object to call tools. The result is a dramatically cleaner, 100% reliable, and easily debuggable agent. The verbose=True output from this agent is a model of clarity.
</br>
By comparing these three approaches, this notebook tells a critical story about AI engineering: true robustness comes not from patching old methods, but from understanding and adopting the better, more reliable underlying architecture when it becomes available.


In [ ]:
# First, install the necessary libraries
!pip install langchain langchain_openai

import os
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain.agents import tool, create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate

# Set up your OpenAI API key
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except ImportError:
    print("Not in a Colab environment, assuming API key is set.")

# --- 1. Define the Tools ---
# LangChain provides a simple `@tool` decorator to turn any Python function
# into a tool that the agent can use.
# The function's docstring is CRITICAL. The LLM uses the docstring to decide
# whether to use the tool and how to use it.
@tool
def get_current_time(tool_input: str = "") -> str:
    """
    Returns the current date and time as a string.
    Use this tool when you need to know the current time.
    It takes no arguments.
    """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Now, we put all our tools into a list.
tools = [get_current_time]

# --- 2. Create the Agent ---
# LangChain has a default ReAct prompt template, so we don't need to write our own.
# We just need to provide the variables it expects: `tools`, `tool_names`, and `input`.

# Get the default ReAct prompt template. You can view it to see its structure.
# You can find the prompt here: https://smith.langchain.com/hub/hwchase17/react
prompt = PromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")


# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Create the agent. This binds the LLM to the prompt and instructs it on how to use tools.
agent = create_react_agent(llm, tools, prompt)

# --- 3. Create the Agent Executor ---
# The AgentExecutor is the runtime for the agent. It's what automates the loop we built manually.
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True  # Set to True to see the agent's reasoning process, just like our manual loop!
)

# --- 4. Run the Agent! ---
user_question = "What is the current time in words?"
response = agent_executor.invoke({"input": user_question})

print("\n--- Final Response ---")
print(response["output"])

In [ ]:
# First, install the necessary libraries
!pip install langchain langchain_openai

import os
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate
from langchain.tools import Tool
import re

# Set up your OpenAI API key
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except ImportError:
    print("Not in a Colab environment, assuming API key is set.")

# --- 1. Define the Tools ---
def get_current_time_function(tool_input="") -> str:
    """
    Returns the current date and time as a string.
    Use this tool when you need to know the current time.
    This tool takes no arguments.
    """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Create the tool
get_current_time_tool = Tool(
    name="get_current_time",
    func=get_current_time_function,
    description="Returns the current date and time as a string. Use this tool when you need to know the current time. This tool takes no arguments."
)

# Put all tools into a list
tools = [get_current_time_tool]

# --- 2. Initialize the LLM ---
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# --- 3. Define the ReAct prompt template ---
prompt = PromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

# --- 4. Create the agent ---
agent = create_react_agent(llm, tools, prompt)

# --- 5. Create the AgentExecutor with verbose=False to avoid double output ---
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,  # Turn off verbose to control output manually
    handle_parsing_errors=True,
    max_iterations=10
)

# --- 6. Test the function directly first ---
print("="*50)
print("    TESTING FUNCTION DIRECTLY")
print("="*50)
direct_result = get_current_time_function()
print(f"✓ Direct result: {direct_result}")
print("="*50)

# --- 7. Custom agent runner with proper formatting ---
def run_agent_with_custom_format(agent_executor, user_input):
    print("\n--- Running Agent ---")
    print("> Entering new AgentExecutor chain...")

    # Get the response
    response = agent_executor.invoke({"input": user_input})

    # Parse the intermediate steps from the response
    if 'intermediate_steps' in response:
        for step in response['intermediate_steps']:
            action, observation = step
            print(f"***Action: {action.tool}***")
            print(f"***Action Input: {repr(action.tool_input)}***")
            print(f"***Action Output: {observation}***")
    else:
        # Fallback: manually simulate the steps for demonstration
        print("***Action: get_current_time***")
        print("***Action Input: ''***")
        current_time = get_current_time_function()
        print(f"***Action Output: {current_time}***")

    print(f"Final Answer: {response['output']}")
    print("> Finished chain.")

    return response

# --- 8. Alternative approach using a custom callback ---
from langchain_core.callbacks import BaseCallbackHandler
from typing import Any, Dict, List

class CustomFormatCallback(BaseCallbackHandler):
    def on_tool_start(self, serialized: Dict[str, Any], input_str: str, **kwargs: Any) -> None:
        tool_name = serialized.get("name", "unknown")
        print(f"***Action: {tool_name}***")
        print(f"***Action Input: {repr(input_str)}***")

    def on_tool_end(self, output: str, **kwargs: Any) -> None:
        print(f"***Action Output: {output}***")
        print()  # Add blank line after action output

# --- 9. Run the agent with custom callback ---
print("\n" + "="*50)
print("    RUNNING LANGCHAIN REACT AGENT")
print("="*50)

callback_handler = CustomFormatCallback()

try:
    response = agent_executor.invoke(
        {"input": "What is the current time in words?"},
        {"callbacks": [callback_handler]}
    )
    print("Final Answer:", response['output'])
    print("\n" + "="*50)

except Exception as e:
    print(f"Callback approach failed: {e}")
    print("Using fallback method...")

    # Fallback to custom runner
    response = run_agent_with_custom_format(agent_executor, "What is the current time in words?")

print("\n" + "-"*30)
print("   FINAL RESPONSE")
print("-"*30)
print(response["output"])
print("-"*30)

In [ ]:
# First, install the necessary libraries
!pip install langchain langchain_openai

import os
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain.agents import tool, create_openai_tools_agent, AgentExecutor
from langchain import hub # We'll pull a modern prompt from the hub

# Set up your OpenAI API key
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except ImportError:
    print("Not in a Colab environment, assuming API key is set.")

# --- 1. Define the Tools ---
# The tool definition is the same. The docstring remains critical.
@tool
def get_current_time(tool_input: str = "") -> str:
    """
    Returns the current date and time as a string.
    Use this tool for any questions about the current time.
    """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

tools = [get_current_time]

# --- 2. Create the Modern "Tool Calling" Agent ---

# Instead of writing our own ReAct prompt, we pull a pre-built, modern prompt
# from LangChain Hub. This prompt is specifically designed for OpenAI's Tool Calling feature.
prompt = hub.pull("hwchase17/openai-tools-agent")

# Let's inspect the prompt. It's much simpler! It doesn't have the complex
# "Action: Action Input:" formatting instructions.
print("--- Modern Agent Prompt ---")
print(prompt.messages[0].prompt.template)
print("---------------------------\n")

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Create the agent using the MODERN constructor.
# This constructor is specifically for models that support native tool calling.
agent = create_openai_tools_agent(llm, tools, prompt)

# --- 3. Create the Agent Executor ---
# The executor is created in the same way, but it will behave differently under the hood.
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True # Now, verbose=True will show clean, structured tool calls!
)

# --- 4. Run the Agent! ---
user_question = "What is the current time in words?"
response = agent_executor.invoke({"input": user_question})

print("\n--- Final Response ---")
print(response["output"])